In [ ]:
import h5py
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from glob import glob
import torch
import torchvision
from torchvision import datasets
from torchvision import transforms
from torchvision.transforms import ToTensor
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torch import nn
from torch.functional import F
from torch.optim import Adam
import numpy as np
from tqdm.auto import tqdm
import umap
import umap.plot
import pandas as pd
from warnings import filterwarnings
import torchvision.utils as vutils
import matplotlib.animation as animation
from IPython.display import HTML
from tqdm import tqdm
import itertools

In [ ]:
directory = f'/home/sarperyn/sarperyurtseven/ProjectFiles/dataset/NIRCAM/'

In [ ]:
h5_files = glob(os.path.join(directory,'**/*160.h5'))

In [ ]:
h5_files

In [ ]:
#data1441 = [h5py.File(h5_path,"r+") for h5_path in h5_files][0]
#data1386 = [h5py.File(h5_path,"r+") for h5_path in h5_files][1]

In [ ]:
#keys = [x for x in [h5py.File(h5_path,"r+") for h5_path in h5_files][0].keys()]

In [ ]:
#data1441[keys[10]]

* rotated_shifted_135
* rotated_shifted_270
* shifted

In [ ]:
filterwarnings('ignore')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
class H5Dataset(Dataset):
    
    def __init__(self,h5_paths):
        
        self.h5_paths  = h5_paths
        self._archives = [h5py.File(h5_path,"r+") for h5_path in self.h5_paths]
        self.keys      = {}
        self.d = {}
        
        idx  = 0
        idx2 = 0
        
        
        for a, archive in enumerate(self.archives):
            
            for i in range(len(archive)):
                
                key = list(archive.keys())[i]  
                self.keys[idx] = (a, key)
                idx += 1
                
                for j in range(archive[key].shape[0]):
                    
                    self.d[idx2] = archive[key][j]
                    idx2 +=1
                    
        self._archive=None
    @property
    def archives(self):
        if self._archives is None: 
            self._archives = [h5py.File(h5_path, "r") for h5_path in self.h5_paths]
        return self._archives
    
    def __len__(self):
        return len(self.d)
    
    def __getitem__(self,index):
        
        data = self.d[index].astype(np.float32)
        data = torch.from_numpy(data)
        
        return data

    

In [ ]:
loader = torch.utils.data.DataLoader(H5Dataset(h5_files),batch_size=16,shuffle=True)
loader.__len__()

In [ ]:
doenum = enumerate(loader)

In [ ]:
idx,data = next(doenum)

In [ ]:
data.shape

In [ ]:
fig, axs = plt.subplots(2, 4)
nth = 0
for i in range(2):
    for j in range(4):
        axs[i, j].imshow(data[nth],clim=(0,50),cmap='gray')
        nth +=1
        
plt.subplots_adjust(wspace=0,hspace=0)
fig.patch.set_facecolor('#423f3b')
plt.show()

# Conv Calculater

In [ ]:
def calculate_conv_dims(input_size,paddings:list,kernels:list,strides:list):
    
    outputs = []
    outputs.append(input_size)
    for i in range(len(paddings)):
        
        output_size = (input_size + (2*paddings[i]) - (kernels[i] - 1) - 1)/strides[i] + 1
        outputs.append(int(output_size))
        input_size = output_size
        
    print(outputs)

In [ ]:
kernels_forward = [5,3,5,3]
paddings_forward= [0,0,0,0]
strides_forward = [2,1,2,1]

In [ ]:
calculate_conv_dims(160,paddings_forward,kernels_forward,strides_forward)

In [ ]:
34*34*32/4

In [ ]:
160*160/2/2

# AE

In [ ]:
class AE(nn.Module):
    
    def __init__(self,latent_dim=200):
        super().__init__()
        self.encoder = nn.Sequential( 
            nn.Linear(25600,12800), # 154x154x20 --> 155x155x16
            nn.LeakyReLU(),
            nn.Linear(12800,6400), # 155x155x16 --> 312x312x16
            nn.LeakyReLU(),
            nn.Linear(6400,1024),# 312x312x16 --> 314x314x8
            nn.LeakyReLU(),
            nn.Linear(1024,512), # 314x314x8 --> 320x320x3
            nn.LeakyReLU(),
            nn.Linear(512,latent_dim),
            nn.LeakyReLU(),
        )
        
        self.latent = nn.Linear(latent_dim,512)
                
        self.decoder = nn.Sequential(
            nn.Linear(512,1024), # 154x154x20 --> 155x155x16
            nn.LeakyReLU(),
            nn.Linear(1024,2048), # 155x155x16 --> 312x312x16
            nn.LeakyReLU(),
            nn.Linear(2048,6400),# 312x312x16 --> 314x314x8
            nn.LeakyReLU(),
            nn.Linear(6400,12800),# 314x314x8 --> 320x320x3
            nn.LeakyReLU(),
            nn.Linear(12800,25600),
        )
        
        
    def forward(self, x):
        
        x = x.view(x.size(0),-1)
        
        x = self.encoder(x)
        
        latent = self.latent(x)
        
        x = self.decoder(latent)
    
        return x,latent

In [ ]:
model = AE()

In [ ]:
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=2e-3)  

In [ ]:
def train(model,train_loader,optimizer,device,EPOCH):
    
    #img_list = []
    losses = []
    iters = 0
    criterion = nn.MSELoss()
    print("Starting Training Loop...")
    with tqdm(total=len(train_loader)*EPOCH) as tt:
        
        model.train()
        
        for epoch in range(EPOCH):
            
            total_loss, batch_count = 0, 0
            
            for idx, batch in enumerate(train_loader):
                
                batch = batch.to(device)
                
                output, latent = model(batch)
                
                loss = criterion(output,batch.view(batch.size(0),-1))
                
                optimizer.zero_grad()
                loss.backward()
                
                total_loss += loss.item()
                batch_count +=1
                tt.update()
                losses.append(loss.item())
                iters += 1
                    
                if iters % 100 == 0:    
                    print('[%d/%d][%d/%d]\t Loss: %.4f\t'
                      % (epoch, EPOCH, idx, len(train_loader),
                         total_loss / batch_count))
                    
                    
        return losses

In [ ]:
train(model=model,
      train_loader=loader,
      optimizer=optimizer,
      device=device,
      EPOCH=100)

In [ ]:
#model1 = torch.load('models/vae_250E_3e-6_128_model.pt')
#model2 = torch.load('models/vae_220E_3e-6_64_model.pt')
#model3 = torch.load('models/vae_500E_3e-6_64_model.pt')

# Visualize Results

In [ ]:
def visualize_results(model,data_loader):
    
    _,images   = next(enumerate(data_loader))
    batch_size = images.size(0) 
    channel_size = 1
    w_h = images.size(2)
    
    if next(model.encoder.parameters()).shape[1] != 3:
        images_flatten = images.view(batch_size, -1)
        output,_ = model(images_flatten.to(device))
    else:
        output,_ = model(images.to(device))
    
    images = images.numpy()
    output = output.view(batch_size, channel_size, w_h, w_h)
    output = output.cpu().detach().numpy()
    
    _, axes = plt.subplots(nrows=2,ncols=8,figsize=(28,8))

    for idx,(row,col) in enumerate(itertools.product(range(2),range(8))):
            
            axes[row][col].imshow(np.squeeze(output[idx]),clim=(0,50),cmap='gray')
            axes[row][col].set_yticks([])
            axes[row][col].set_xticks([])

In [ ]:
visualize_results(model,loader)

# Generate Images

In [ ]:
def generate_random_imgs_conv(model,nsample):
    
    rand_features = torch.randn(nsample, 500).to(device)
    x = model.linear(rand_features)
    x = x.view(12,20,154,154)
    rand_generated = model.decoder(x)
    
    images = rand_generated.view(nsample,1,320,320)
    images = images.cpu().detach().numpy()
    
    for i in range(12):
            plt.subplot(3,4,i+1)
            plt.tight_layout()
            plt.imshow(np.transpose((images[i]*255).astype(np.uint8),(1, 2, 0)),cmap='gray')
            plt.xticks([])
            plt.yticks([])

In [ ]:
generate_random_imgs_conv(vae_model,12)

# Latent Space Visualization

In [ ]:
def visualize2d_interactive(data,targets,classes):
    
    hover_data = pd.DataFrame({'index':np.arange(len(data)),
                               'label':targets})
    
    hover_data['item'] = hover_data.label.map(classes)
    mapping = umap.UMAP().fit(data)
    umap.plot.output_notebook()
    p = umap.plot.interactive(mapping, labels=targets, hover_data=hover_data, point_size=2,)
    umap.plot.show(p)
    

In [ ]:
def visualize2d(data,targets):
    
    mapping = umap.UMAP().fit(data)
    umap.plot.points(mapping, theme='fire');

In [ ]:
#torch.save(vae_model,'noisy_250.pt')
vae_model = torch.load('noisy_250.pt')

In [ ]:
vae_model

In [ ]:
z_mean, z_log_var = vae_model.mean(x), vae_model.log_var(x)
encoded = vae_model.reparameterize(z_mean,z_log_var)
visualize2d(encoded.cpu().detach().numpy(),targets='')

# Regenerate

In [ ]:
h5py.File(h5_files[1],"r").keys()
regenerate = np.array(h5py.File(h5_files[1],"r")['1386_HIP-68245_F300M_MASKA335R_sci'])[3:4]
original = np.squeeze(regenerate)

In [ ]:
data = regenerate.astype(np.float32)
data = torch.from_numpy(data)
data = data.view(1,1,320,320).to(device)
_,_,_,regenerated = vae_model(data)
result = np.squeeze(regenerated.cpu().detach().numpy())

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, sharex=True, sharey=True, figsize=(25,10))

for images, row in zip([result,original],axes):
    
    row.imshow(images,cmap='gray')
    row.get_xaxis().set_visible(False)
    row.get_yaxis().set_visible(False)